# Create Slice for Fabric Rack
This is the slice creation for the ```RTT_Window_Size``` tutorial, This is the first step of the assignment, to create the slice on fabric using fablib
 

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

try:
    fablib = fablib_manager()
                         
    fablib.show_config()
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
slice_name = "RttWindowTCP"
site = fablib.get_random_site()
print(f'site = {site}')

nicmodel = "NIC_Basic"
image = "default_ubuntu_20"

cores = 1
ram = 4
disk = 100

In [ ]:
try:
    # Create a slice
    slice = fablib.new_slice(name=slice_name)
    #Server
    Server = slice.add_node(name="Server", site=site)
    Server.set_capacities(cores=cores, ram=ram, disk=disk)
    Server.set_image(image)
    S = Server.add_component(model=nicmodel, name="Server_P").get_interfaces()[0] 
    
    
    # Client
    Client = slice.add_node(name="Client", site=site)
    Client.set_capacities(cores=cores, ram=ram, disk=disk)
    Client.set_image(image)
    C = Client.add_component(model = nicmodel, name="Client_P").get_interfaces()[0] 

    LAN_CONN = slice.add_l2network(name="LAN_CONN", interfaces=[S, C])
    
    # Submit the slice
    slice.submit()
except Exception as e:
    print(f"Exception: {e}")

## Continue to The experiment notebook

Once You have completed this notebook you should be able to continue to the RTT Window size Notebook, you can ether open it on the explorer or click [Here](./RTT_Window_Size.ipynb) to open the next notebook.